In [3]:
import sys
sys.path.append("./src")
import VCSM
import SKCM_dataset


import numpy as np
from matplotlib import pyplot as plt

from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest
from sksurv.ensemble import GradientBoostingSurvivalAnalysis

from mvlearn.embed import MCCA

from lifelines.utils import concordance_index
from monai.networks.nets import FullyConnectedNet

from torch import optim
import torch
from torch import Tensor

In [4]:
lr = 1e-4
encoder_layer = [128, 64, 32]
predictor_layer = []
alpha = 0.
beta = 10.
gamma1 = 0.
gamma2 = 0.
top_K_img = 250
top_K_RNA = 500
top_K_meth = 500
top_K_miRNA = 50

C_idx = []

for seed in range(10):    
    X_train, D_train, E_train, X_test, D_test, E_test = SKCM_dataset.load_data(
    seed = seed, 
    top_K_img = top_K_img, top_K_RNA = top_K_RNA, top_K_meth = top_K_meth, top_K_miRNA = top_K_miRNA)

    X_test = [XXX.cuda() for XXX in X_test]
    
    filename = "./models/VCSM_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}_{}".format(
        encoder_layer, predictor_layer, alpha, beta, gamma1, gamma2, 
        lr, top_K_img, top_K_RNA, top_K_meth, top_K_miRNA, seed)

    
    model = VCSM.CoxModel(
        D_dat=[x.shape[1] for x in X_train], 
        encoder_var_layer=encoder_layer, decoder_layer=predictor_layer,
        N_validate=1, alpha=alpha, beta=beta, gamma1=gamma1, gamma2=gamma2,
        filename=filename,
        silent=True, 
        lr=lr
        ).cuda()

    model.load_state_dict(torch.load(filename, weights_only = True))
    model.eval()


    C = model.compute_C(X_test, D_test, E_test)
    print("seed {}, Concordance Score: {:.3}".format(seed, C))
    C_idx.append(C)

print("\nCorcordance Score: {:.3} ({:.3})".format( np.mean(C_idx), np.std(C_idx) ))

seed 0, Concordance Score: 0.448
seed 1, Concordance Score: 0.64
seed 2, Concordance Score: 0.765
seed 3, Concordance Score: 0.862
seed 4, Concordance Score: 0.8
seed 5, Concordance Score: 0.885
seed 6, Concordance Score: 0.852
seed 7, Concordance Score: 0.778
seed 8, Concordance Score: 0.765
seed 9, Concordance Score: 0.667

Corcordance Score: 0.746 (0.124)
